In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score
import scipy.io
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv("../data/data.csv")
df

,id,author,geometry,pressure [MPa],mass_flux [kg/m2-s],x_e_out [-],D_e [mm],D_h [mm],length [mm],chf_exp [MW/m2]
0,0,Thompson,tube,7.00,3770.0,0.1754,NaN,10.8,432.0,3.6
1,1,Thompson,tube,NaN,6049.0,-0.0416,10.3,10.3,762.0,6.2
2,2,Thompson,NaN,13.79,2034.0,0.0335,7.7,7.7,457.0,2.5
3,3,Beus,annulus,13.79,3679.0,-0.0279,5.6,15.2,2134.0,3.0
4,4,NaN,tube,13.79,686.0,NaN,11.1,11.1,457.0,2.8
...,...,...,...,...,...,...,...,...,...,...
31639,31639,Thompson,NaN,NaN,1736.0,0.0886,NaN,7.8,591.0,2.3
31640,31640,NaN,NaN,13.79,NaN,NaN,4.7,4.7,NaN,3.9
31641,31641,Thompson,NaN,18.27,658.0,-0.1224,3.0,3.0,150.0,2.3
31642,31642,Thompson,tube,6.89,3825.0,NaN,23.6,23.6,1972.0,3.7


In [3]:
# Definir un diccionario de mapeo para los valores conocidos de "author" y "geometry"
mapeo_geometry = {
    'Thompson': 'tube',
    'Peskov': 'tube',
    'Weatherhead': 'tube',
    'Inasaka': 'tube',
    'Williams': 'tube',
    'Beus': 'annulus',
    'Janssen': 'annulus',
    'Mortimore': 'annulus',
    'Richenderfer': 'plate',
    'Kossolapov': 'plate'
}

# Rellenar los valores nulos en "geometry" utilizando el diccionario de mapeo
df['geometry'] = df['geometry'].fillna(df['author'].map(mapeo_geometry))

In [4]:
# Llenar los nulos en "length [mm]" con 10.0 cuando la columna "geometry" es "plate"
df.loc[df['geometry'] == 'plate', 'length [mm]'] = df.loc[df['geometry'] == 'plate', 'length [mm]'].fillna(10.0)

# Llenar los nulos en "D_e [mm]" con 15.0 cuando la columna "geometry" es "plate"
df.loc[df['geometry'] == 'plate', 'D_e [mm]'] = df.loc[df['geometry'] == 'plate', 'D_e [mm]'].fillna(15.0)

# Llenar los nulos en "D_h [mm]" con 120.0 cuando la columna "geometry" es "plate"
df.loc[df['geometry'] == 'plate', 'D_h [mm]'] = df.loc[df['geometry'] == 'plate', 'D_h [mm]'].fillna(120.0)


In [5]:
# Filtrar las filas con valores nulos en la columna 'x_e_out'
df_nulos = df[df['x_e_out [-]'].isnull()]

# Crear un nuevo DataFrame con las filas que contienen valores nulos en 'x_e_out'
nuevos_datos = df_nulos.copy()

In [6]:
df = df.dropna(subset=['x_e_out [-]'])

# Imprimir el DataFrame original sin las filas con valores nulos

In [7]:
df['geometry'] = df['geometry'].fillna('tube')

In [8]:
# Definir un diccionario de mapeo para los valores conocidos de "author" y "geometry"
mapeo_geometry = {
    'Thompson': 'tube',
    'Peskov': 'tube',
    'Weatherhead': 'tube',
    'Inasaka': 'tube',
    'Williams': 'tube',
    'Beus': 'annulus',
    'Janssen': 'annulus',
    'Mortimore': 'annulus',
    'Richenderfer': 'plate',
    'Kossolapov': 'plate'
}

# Rellenar los valores nulos en "geometry" utilizando el diccionario de mapeo
nuevos_datos['geometry'] = nuevos_datos['geometry'].fillna(nuevos_datos['author'].map(mapeo_geometry))

In [9]:
nuevos_datos['geometry'] = nuevos_datos['geometry'].fillna('tube')

In [10]:
df.loc[df['geometry'] == 'tube', 'author'] = df.loc[df['geometry'] == 'tube', 'author'].fillna('Thompson')
df.loc[df['geometry'] == 'plate', 'author'] = df.loc[df['geometry'] == 'plate', 'author'].fillna('Richenderfer')
df.loc[df['geometry'] == 'annulus', 'author'] = df.loc[df['geometry'] == 'annulus', 'author'].fillna('Beus')

In [11]:
nuevos_datos.loc[nuevos_datos['geometry'] == 'tube', 'author'] = nuevos_datos.loc[nuevos_datos['geometry'] == 'tube', 'author'].fillna('Thompson')
nuevos_datos.loc[nuevos_datos['geometry'] == 'plate', 'author'] = nuevos_datos.loc[nuevos_datos['geometry'] == 'plate', 'author'].fillna('Richenderfer')
nuevos_datos.loc[nuevos_datos['geometry'] == 'annulus', 'author'] = nuevos_datos.loc[nuevos_datos['geometry'] == 'annulus', 'author'].fillna('Beus')

In [12]:
# Aplicar codificación one-hot a la columna "author"
df = pd.get_dummies(df, columns=['author'], prefix=['author'])

# Aplicar codificación one-hot a la columna "geometry"
df = pd.get_dummies(df, columns=['geometry'], prefix=['geometry'])

# Aplicar codificación one-hot a la columna "author"
nuevos_datos = pd.get_dummies(nuevos_datos, columns=['author'], prefix=['author'])

# Aplicar codificación one-hot a la columna "geometry"
nuevos_datos = pd.get_dummies(nuevos_datos, columns=['geometry'], prefix=['geometry'])

In [13]:
# Dividir en X e y
X = df.drop(['x_e_out [-]', 'id'], axis=1)  # Todas las columnas excepto 'x_e_out'
y = df['x_e_out [-]']  # Solo la columna 'x_e_out

In [14]:
# Separar las características (X) y la columna objetivo (y)
X_nuevos_datos = nuevos_datos.drop(columns=["x_e_out [-]"])
y_nuevos_datos = nuevos_datos["x_e_out [-]"]

In [15]:
X.rename(columns={'pressure [MPa]': 'pressure', 'mass_flux [kg/m2-s]': 'mass_flux', 
                    'D_e [mm]': 'D_e', 'D_h [mm]': 'D_h', 'length [mm]': 'length'
                    , 'chf_exp [MW/m2]': 'chf_exp'}, inplace=True)

y.name = 'x_e_out'

In [16]:
X_nuevos_datos.rename(columns={'pressure [MPa]': 'pressure', 'mass_flux [kg/m2-s]': 'mass_flux', 
                    'D_e [mm]': 'D_e', 'D_h [mm]': 'D_h', 'length [mm]': 'length'
                    , 'chf_exp [MW/m2]': 'chf_exp'}, inplace=True)


In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
df_additional = pd.read_csv('data/Data_CHF_Zhao_2020_ATE.csv')

In [19]:
# Aplicar codificación one-hot a la columna "author"
df_additional = pd.get_dummies(df_additional, columns=['author'], prefix=['author'])

# Aplicar codificación one-hot a la columna "geometry"
df_additional = pd.get_dummies(df_additional, columns=['geometry'], prefix=['geometry'])

In [20]:
# Dividir en X e y
X_additional = df_additional.drop(['x_e_out [-]', 'id'], axis=1)  # Todas las columnas excepto 'x_e_out'
Y_additional = df_additional['x_e_out [-]']  # Solo la columna 'x_e_out

In [21]:
X_additional.rename(columns={'pressure [MPa]': 'pressure', 'mass_flux [kg/m2-s]': 'mass_flux', 
                    'D_e [mm]': 'D_e', 'D_h [mm]': 'D_h', 'length [mm]': 'length'
                    , 'chf_exp [MW/m2]': 'chf_exp'}, inplace=True)

Y_additional.name = 'x_e_out'

In [22]:
# Paso 4: Concatenar X_additional y y_additional a X_train y y_train
X_train = np.concatenate((X_train, X_additional), axis=0)
y_train = np.concatenate((y_train, Y_additional), axis=0)


In [23]:
from sklearn.impute import KNNImputer

# Paso 5: Crear el imputador KNN
imputer = KNNImputer(n_neighbors=5)

# Paso 6: Imputar X_train
X_train = imputer.fit_transform(X_train)

# Paso 7: Imputar X_test utilizando la misma imputación aprendida de X_train
X_test = imputer.transform(X_test)

c:\Python311\Lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNNImputer was fitted without feature names
  warnings.warn(


In [24]:
# Índices de columnas a eliminar
column_indices_to_drop = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

# Eliminar columnas de X_train
X_train = np.delete(X_train, column_indices_to_drop, axis=1)

# Eliminar columnas de X_test
X_test = np.delete(X_test, column_indices_to_drop, axis=1)

In [25]:
X_nuevos_datos = X_nuevos_datos.drop(columns=["id"])

In [38]:
y_nuevos_datos.info()

<class 'pandas.core.series.Series'>
Int64Index: 10415 entries, 4 to 31642
Series name: x_e_out [-]
Non-Null Count  Dtype  
--------------  -----  
0 non-null      float64
dtypes: float64(1)
memory usage: 162.7 KB


In [26]:


# #Separar las características (X) y la columna objetivo (y)
# X_nuevos_datos = X_nuevos_datos.drop(columns=["x_e_out"])
# y_nuevos_datos = nuevos_datos["x_e_out"]

# Imputar los valores nulos en las características (X)

X_nuevos_datos = imputer.transform(X_nuevos_datos)




# Convertir el resultado a DataFrame
#X_nuevos_datos = pd.DataFrame(X_nuevos_datos, columns=X_nuevos_datos.columns)

c:\Python311\Lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNNImputer was fitted without feature names
  warnings.warn(


In [27]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import KNNImputer



scaler = StandardScaler()
model = XGBRegressor(colsample_bytree=0.5, learning_rate=0.01, max_depth=5, n_estimators=1000, subsample=0.6)

# Paso 6: Crear el pipeline
xg_pipeline = Pipeline([
    ('scaler', scaler),
    ('model', model)
])

# Paso 7: Ajustar el pipeline a los datos de entrenamiento
xg_pipeline.fit(X_train, y_train)

# Paso 8: Realizar predicciones en los datos de prueba
y_pred = xg_pipeline.predict(X_test)

from sklearn.metrics import mean_squared_error
import math

# Calcular el error cuadrático medio (MSE)
mse = mean_squared_error(y_test, y_pred)

# Calcular el RMSE
rmse = math.sqrt(mse)

print("RMSE:", rmse)

RMSE: 0.07405480054626608


In [41]:
#Separar las características (X) y la columna objetivo (y)
X_nuevos_datos = nuevos_datos.drop(columns=["x_e_out [-]"])
y_nuevos_datos = nuevos_datos["x_e_out [-]"]

# Imputar los valores nulos en las características (X)
imputer = KNNImputer(n_neighbors=5)
X_nuevos_datos_imputed = imputer.fit_transform(X_nuevos_datos)

# Convertir el resultado a DataFrame
X_nuevos_datos_imputed = pd.DataFrame(X_nuevos_datos_imputed, columns=X_nuevos_datos.columns)

# # Crear el objeto de regresión lineal
# regression_model = LinearRegression()


In [42]:
X_nuevos_datos_imputed = X_nuevos_datos_imputed.drop(columns=["id"])

In [43]:

# Realizar predicciones en la columna objetivo (y) para los datos imputados
y_pred1 = xg_pipeline.predict(X_nuevos_datos_imputed)

# Combinar las predicciones con los datos originales
nuevos_datos["x_e_out [-]"] = y_pred1

# Los datos ahora tienen las predicciones en la columna objetivo (y)

# Guardar los datos en un archivo para la competición de Kaggle
nuevos_datos.to_csv("nuevos_datos_con_predicciones2.csv", index=False)

c:\Python311\Lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


ValueError: X has 19 features, but StandardScaler is expecting 6 features as input.

In [48]:
# # Leer el archivo CSV
prueba1 = pd.read_csv("nuevos_datos_con_predicciones2.csv")

# Mantener solo las columnas "id" y "x_e_out [mm]"
columnas_deseadas = ["id", "x_e_out [-]"]
prueba1 = prueba1[columnas_deseadas]

# Guardar el DataFrame resultante en un nuevo archivo CSV
prueba1.to_csv("nuevos_datos_con_predicciones_filtrado10.csv", index=False)

In [28]:
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import math

# Paso 5: Definir las transformaciones y los modelos
scaler = StandardScaler()
rf_model = RandomForestRegressor(max_depth= 12, min_samples_leaf= 5, min_samples_split= 10, n_estimators= 100)
gb_model = GradientBoostingRegressor(max_depth=5, n_estimators=100, learning_rate=0.1)
ada_model = AdaBoostRegressor(n_estimators=100)
et_model = ExtraTreesRegressor(min_samples_split=8, n_estimators= 400)

# Paso 6: Crear los pipelines para Random Forest y Gradient Boosting
rf_pipeline = Pipeline([
    ('scaler', scaler),
    ('model', rf_model)
])

gb_pipeline = Pipeline([
    ('scaler', scaler),
    ('model', gb_model)
])

ada_pipeline = Pipeline([
    ('scaler', scaler),
    ('model', ada_model)
])

et_pipeline = Pipeline([
    ('scaler', scaler),
    ('model', et_model)
])

# Paso 7: Ajustar los pipelines a los datos de entrenamiento
rf_pipeline.fit(X_train, y_train)
gb_pipeline.fit(X_train, y_train)
ada_pipeline.fit(X_train, y_train)
et_pipeline.fit(X_train, y_train)

# Paso 8: Realizar predicciones en los datos de prueba
y_pred_rf = rf_pipeline.predict(X_test)
y_pred_gb = gb_pipeline.predict(X_test)
y_pred_ada = ada_pipeline.predict(X_test)
y_pred_et = et_pipeline.predict(X_test)

# Calcular el error cuadrático medio (MSE) y el RMSE para Random Forest
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = math.sqrt(mse_rf)
print("RMSE (Random Forest):", rmse_rf)

# Calcular el error cuadrático medio (MSE) y el RMSE para Gradient Boosting
mse_gb = mean_squared_error(y_test, y_pred_gb)
rmse_gb = math.sqrt(mse_gb)
print("RMSE (Gradient Boosting):", rmse_gb)

# Calcular el error cuadrático medio (MSE) y el RMSE para AdaBoost
mse_ada = mean_squared_error(y_test, y_pred_ada)
rmse_ada = math.sqrt(mse_ada)
print("RMSE (AdaBoost):", rmse_ada)

# Calcular el error cuadrático medio (MSE) y el RMSE para Extra Trees
mse_et = mean_squared_error(y_test, y_pred_et)
rmse_et = math.sqrt(mse_et)
print("RMSE (Extra Trees):", rmse_et)

RMSE (Random Forest): 0.0756613423620883
RMSE (Gradient Boosting): 0.07479308216597548
RMSE (AdaBoost): 0.10073675363448692
RMSE (Extra Trees): 0.07853953066069258


In [39]:
from sklearn.ensemble import VotingRegressor
voting_reg1 = VotingRegressor([('XG', xg_pipeline), ('GraBoosst', gb_pipeline)], 
                              weights=[16/20, 4/20])
voting_reg1.fit(X_train, y_train)

VotingRegressor(estimators=[('XG',
                             Pipeline(steps=[('scaler', StandardScaler()),
                                             ('model',
                                              XGBRegressor(base_score=None,
                                                           booster=None,
                                                           callbacks=None,
                                                           colsample_bylevel=None,
                                                           colsample_bynode=None,
                                                           colsample_bytree=0.5,
                                                           early_stopping_rounds=None,
                                                           enable_categorical=False,
                                                           eval_metric=None,
                                                           feature_types=None,
                                                           gamma=None,
                                                           gpu_id=None,
                                                           grow_policy=None,
                                                           importance_t...
                                                           max_cat_to_onehot=None,
                                                           max_delta_step=None,
                                                           max_depth=5,
                                                           max_leaves=None,
                                                           min_child_weight=None,
                                                           missing=nan,
                                                           monotone_constraints=None,
                                                           n_estimators=1000,
                                                           n_jobs=None,
                                                           num_parallel_tree=None,
                                                           predictor=None,
                                                           random_state=None, ...))])),
                            ('GraBoosst',
                             Pipeline(steps=[('scaler', StandardScaler()),
                                             ('model',
                                              GradientBoostingRegressor(max_depth=5))]))],
                weights=[0.8, 0.2])

In [40]:
ensamble1=voting_reg1.predict(X_test)


mse = mean_squared_error(y_test, ensamble1)
r2 = r2_score(y_test, ensamble1)

print("Mean Squared Error (MSE): ", mse)
print("R-squared (R2) Score: ", r2)
from math import sqrt

#Calcular el RMSE

rmse = sqrt(mse)
print("Root Mean Squared Error (RMSE): ", rmse)

Mean Squared Error (MSE):  0.0054899730897886405
R-squared (R2) Score:  0.4595203212736271
Root Mean Squared Error (RMSE):  0.07409435261737995


In [66]:

# Realizar predicciones en la columna objetivo (y) para los datos imputados
y_pred1 = voting_reg1.predict(X_nuevos_datos_filtered)

# Combinar las predicciones con los datos originales
nuevos_datos["x_e_out [-]"] = y_pred1

# Los datos ahora tienen las predicciones en la columna objetivo (y)

# Guardar los datos en un archivo para la competición de Kaggle
nuevos_datos.to_csv("nuevos_datos_con_predicciones2.csv", index=False)

ValueError: X has 19 features, but StandardScaler is expecting 6 features as input.

In [78]:
# # Leer el archivo CSV
prueba1 = pd.read_csv("nuevos_datos_con_predicciones2.csv")

# Mantener solo las columnas "id" y "x_e_out [mm]"
columnas_deseadas = ["id", "x_e_out [-]"]
prueba1 = prueba1[columnas_deseadas]

# Guardar el DataFrame resultante en un nuevo archivo CSV
prueba1.to_csv("nuevos_datos_con_predicciones_filtrado11.csv", index=False)

In [93]:
from sklearn.impute import KNNImputer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import math

# Paso 5: Definir las transformaciones y el modelo
imputer = KNNImputer(n_neighbors=5)
scaler = StandardScaler()
et_model = ExtraTreesRegressor()

# Paso 6: Crear el pipeline
et_pipeline = Pipeline([
    ('imputer', imputer),
    ('scaler', scaler),
    ('model', et_model)
])

# Paso 7: Definir los parámetros de búsqueda en malla
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 5, 10],
    'model__min_samples_split': [2, 4, 6]
}

# Paso 8: Realizar la búsqueda en malla con validación cruzada
grid_search = GridSearchCV(et_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Paso 9: Realizar predicciones en los datos de prueba con el mejor modelo
y_pred = best_model.predict(X_test)

# Calcular el error cuadrático medio (MSE) y el RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
print("RMSE:", rmse)

Mejores hiperparámetros: {'model__max_depth': None, 'model__min_samples_split': 6, 'model__n_estimators': 300}


c:\Python311\Lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but KNNImputer was fitted without feature names
  warnings.warn(


RMSE: 0.0810616503928035


In [ ]:
from sklearn.impute import KNNImputer
from sklearn.ensemble import AdaBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import math

# Paso 5: Definir las transformaciones y el modelo
imputer = KNNImputer(n_neighbors=5)
scaler = StandardScaler()
ada_model = AdaBoostRegressor()

# Paso 6: Crear el pipeline
ada_pipeline = Pipeline([
    ('imputer', imputer),
    ('scaler', scaler),
    ('model', ada_model)
])

# Paso 7: Definir los parámetros de búsqueda en malla
param_grid = {
    'model__n_estimators': [50, 100, 200],
    'model__learning_rate': [0.01, 0.1, 1.0],
}

# Paso 8: Realizar la búsqueda en malla con validación cruzada
grid_search = GridSearchCV(ada_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Mejores hiperparámetros encontrados
best_params = grid_search.best_params_
print("Mejores hiperparámetros:", best_params)

# Mejor modelo encontrado
best_model = grid_search.best_estimator_

# Paso 9: Realizar predicciones en los datos de prueba con el mejor modelo
y_pred = best_model.predict(X_test)

# Calcular el error cuadrático medio (MSE) y el RMSE
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
print("RMSE:", rmse)

In [31]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# Paso 1: Convertir X_nuevos_datos a DataFrame de Pandas
X_nuevos_datos_df = pd.DataFrame(X_nuevos_datos)

# Paso 2: Imputar valores nulos en X_nuevos_datos utilizando X_train
imputer = KNNImputer(n_neighbors=5)
X_nuevos_datos_imputed = imputer.fit_transform(X_nuevos_datos_df, X_train)

# # Paso 3: Escalar X_nuevos_datos_imputed
# scaler = StandardScaler()
# X_nuevos_datos_scaled = scaler.fit_transform(X_nuevos_datos_imputed)

# Paso 4: Crear DataFrame de X_nuevos_datos_filtered sin las columnas específicas
columns_to_drop = X_nuevos_datos_df.columns[6:19]
X_nuevos_datos_filtered = pd.DataFrame(X_nuevos_datos_imputed, columns=X_nuevos_datos_df.columns)
X_nuevos_datos_filtered = X_nuevos_datos_filtered.drop(columns=columns_to_drop, errors='ignore')
X_nuevos_datos_filtered



,0,1,2,3,4,5
0,13.790,686.0,11.1,11.1,457.0,2.8
1,18.000,750.0,10.0,10.0,1650.0,2.2
2,12.414,2761.2,1.9,1.9,152.0,3.2
3,6.890,7500.0,12.0,12.8,1930.0,4.8
4,15.510,1355.0,5.6,15.2,2134.0,2.1
...,...,...,...,...,...,...
10410,11.030,3800.4,11.5,11.5,1270.0,2.0
10411,1.010,2000.0,15.0,120.0,10.0,6.2
10412,13.790,688.0,11.1,11.1,457.0,2.3
10413,13.790,3933.2,4.7,4.7,315.4,3.9


In [32]:

# Obtener las columnas a mantener en X_nuevos_datos_filtered
columns_to_keep = ['pressure', 'mass_flux', 'D_e', 'D_h', 'length', 'chf_exp']

# Asignar los nombres de columna a X_nuevos_datos_filtered
X_nuevos_datos_filtered.columns = columns_to_keep

X_nuevos_datos_filtered

,pressure,mass_flux,D_e,D_h,length,chf_exp
0,13.790,686.0,11.1,11.1,457.0,2.8
1,18.000,750.0,10.0,10.0,1650.0,2.2
2,12.414,2761.2,1.9,1.9,152.0,3.2
3,6.890,7500.0,12.0,12.8,1930.0,4.8
4,15.510,1355.0,5.6,15.2,2134.0,2.1
...,...,...,...,...,...,...
10410,11.030,3800.4,11.5,11.5,1270.0,2.0
10411,1.010,2000.0,15.0,120.0,10.0,6.2
10412,13.790,688.0,11.1,11.1,457.0,2.3
10413,13.790,3933.2,4.7,4.7,315.4,3.9


In [35]:
# Realizar predicciones en la columna objetivo (y) para los datos imputados
y_pred1 = voting_reg1.predict(X_nuevos_datos_filtered)

# Combinar las predicciones con los datos originales
nuevos_datos["x_e_out [-]"] = y_pred1

# Los datos ahora tienen las predicciones en la columna objetivo (y)

# Guardar los datos en un archivo para la competición de Kaggle
nuevos_datos.to_csv("nuevos_datos_con_predicciones2.csv", index=False)


c:\Python311\Lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [36]:
# # Leer el archivo CSV
prueba1 = pd.read_csv("nuevos_datos_con_predicciones2.csv")

# Mantener solo las columnas "id" y "x_e_out [mm]"
columnas_deseadas = ["id", "x_e_out [-]"]
prueba1 = prueba1[columnas_deseadas]

# Guardar el DataFrame resultante en un nuevo archivo CSV
prueba1.to_csv("nuevos_datos_con_predicciones_filtrado12.csv", index=False)